In [1]:
import pandas as pd
from nltk.corpus import stopwords

ModuleNotFoundError: No module named 'pandas'

In [ ]:
df = pd.read_csv()

In [3]:
def len_title(df):
    len_list = []
    for title in df['Transcript']:
        wordList = title.split()
        len_list.append(len(wordList))
    df['Length'] = len_list

In [8]:
len_title("hello,  there")

2

In [17]:
import re #regex library used for substituting words

In [18]:
#removing stopwords and urls, lemmatization, making lowercase 
# stop_words = stopwords.words('english')
# lemmatizer = WordNetLemmatizer()

# def cleaning(text, stem=False):
#     text = str(text).lower().strip()
#     tokens = []
#     for token in text.split():
#         #Substituting multiple spaces with single space
#         token = re.sub(r'\s+', ' ', token, flags=re.I)
#         if token not in stop_words:
#             if stem:
#                 tokens.append(lemmatizer.lemmatize(token))
#             else:
#                 tokens.append(token)
#     return tokens

NameError: name 'stopwords' is not defined

In [ ]:
def freq_words(df):
    word_dict = {}
    for title in df['Transcript']:
        wordList = title.split()
        for word in wordList:
            if word in word_dict.keys():
                word_dict[word] += 1
            else:
                word_dict[word] = 1
    #find five words with the most frequency

In [ ]:
def freq_punc(df):
    punc_dict = {}
    for title in df['Transcript']:
        